In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import grad
import torch.nn.functional as F
import pandas as pd
import torch.nn.utils.parametrize as parametrize
from torch.nn.functional import normalize
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import r2_score, mean_squared_error
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
start = 1
data = pd.read_csv("data-B90102-30m.csv")
room = data["room_temp"]
airflow = data["airflow_cur"]
supply = data["supply_temp"]

In [ ]:
delt =  np.array(room[start:-1]) - np.array(room[start-1:-2])
delt_supply = np.array(supply[start+1:]) - np.array(supply[start:-1])
delt_air = np.array(airflow[start+1:]) - np.array(airflow[start:-1])

temp_cur = np.array(room[start:-1]).reshape(-1,1)
temp = np.array(room[start:-1]).reshape(-1,1)
output = np.array(room[start+1:]).reshape(-1,1)
air = np.array(airflow[start+1:]).reshape(-1,1)
supply = np.array(supply[start+1:]).reshape(-1,1)

air_ori = torch.tensor(air).to(device).reshape(-1,1).float()
supply_ori = torch.tensor(supply).to(device).reshape(-1,1).float()
temp_ori = torch.tensor(temp).to(device).reshape(-1,1).float()
output_ori = torch.tensor(output).to(device).reshape(-1,1).float()
temp_cur = torch.tensor(temp_cur).to(device).reshape(-1,1).float()

delt = torch.tensor(delt).reshape(-1,1).float().to(device)/(max(temp_ori) - min(temp_ori))
delt_supply = torch.tensor(delt_supply).reshape(-1,1).float().to(device)/(max(supply_ori) - min(supply_ori))
delt_air = torch.tensor(delt_air).reshape(-1,1).float().to(device)/(max(air_ori)- min(air_ori))
delt = torch.concat((delt, delt_supply, delt_air), dim =1)

In [ ]:
air = (air_ori - min(air_ori))/(max(air_ori)- min(air_ori))
air = air.requires_grad_(True)
supply = (supply_ori - min(supply_ori))/(max(supply_ori) - min(supply_ori))
supply = supply.requires_grad_(True)
temp = (temp_ori - min(temp_ori))/(max(temp_ori) - min(temp_ori))
temp = temp.requires_grad_(True)
output = (output - min(output))/(max(output) - min(output))
output = torch.tensor(output).reshape(-1,1).float().to(device)
temp_cur = (temp_cur - min(temp_ori))/(max(temp_ori) - min(temp_ori))

In [ ]:
def data_split(N: int, N_end: int, temp_cur: torch.Tensor, temp: torch.Tensor, supply: torch.Tensor, air: torch.Tensor, output: torch.Tensor, delt: torch.Tensor):
    temp_cur_tr = temp_cur[N:N_end]
    temp_tr = temp[N:N_end]
    supply_tr = supply[N:N_end]
    air_tr = air[N:N_end]
    output_tr = output[N:N_end]
    delt_tr = delt[N:N_end].requires_grad_(True)
    return temp_cur_tr, temp_tr, supply_tr, air_tr, output_tr, delt_tr


In [7]:
class Base_Net(nn.Module):
    def __init__(self):
        super(Base_Net, self).__init__()
        self.fc1 = nn.Linear(3, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x, y, z):
        x = torch.cat((x, y, z), dim = 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model_Base = Base_Net().to(device)

In [ ]:
def train_model(model, optimizer, criterion, temp_tr, supply_tr, air_tr, output_tr, delt_tr, scheduler=None, epochs=20000, patience=1000, threshold=1e-10, print_interval=5000):
    prev_loss = float('inf')
    no_improvement_counter = 0
    loss_array = []
    last_loss = 0
    for epoch in range(epochs):
        model.train()  
        optimizer.zero_grad()
  
        output = model(temp_tr, supply_tr, air_tr)
        loss = criterion(output, output_tr)
        loss_array.append(loss.item())
        loss.backward(retain_graph=True)
        optimizer.step()
        last_loss = loss
        if scheduler:
            scheduler.step()
        if epoch % print_interval == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.9f}')
        if abs(prev_loss - loss.item()) < threshold:
            no_improvement_counter += 1
        else:
            no_improvement_counter = 0  
        prev_loss = loss.item()
        if no_improvement_counter >= patience:
            print(f'Early stopping at epoch {epoch+1} due to no significant improvement.')
            break
        if loss.item() < 0.00000000087022:
            print(f'Early stopping at epoch {epoch+1} due to very low loss.')
            print("min of loss function is", min(loss_array))
            break
    return loss.item()

In [ ]:
epochs = 25000
patience = 1000
threshold = 1e-15
decay_factor = 1.0  
decay_step_size = 1000  
N_start, N_end = 360, 540
Ns_t, Ne_t = 950, 1050
Ns_val, Ne_val = 800, 900
temp_cur_tr, temp_tr, supply_tr, air_tr, output_tr, delt_tr = data_split(N_start, N_end, temp_cur, temp, supply, air, output, delt)
temp_cur_t, temp_t, supply_t, air_t, output_t, delt_t = data_split(Ns_t, Ne_t, temp_cur, temp, supply, air, output, delt)
temp_cur_val, temp_val, supply_val, air_val, output_val, delt_val = data_split(Ns_val, Ne_val, temp_cur, temp, supply, air, output, delt)

In [ ]:
learning = np.arange(0.001, 0.1, 0.002)
num_t = 1
current_mse_score = 0
mse = []
mse.append(10.0)
prev_loss = 10
for lr in learning:
    model_Base = Base_Net().to(device)
    optimizer = optim.Adam(model_Base.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=decay_step_size, gamma=decay_factor)

    loss_Tay = train_model(
        model=model_Base,
        optimizer=optimizer,
        criterion=nn.MSELoss(),
        temp_tr=temp_tr,
        supply_tr=supply_tr,
        air_tr=air_tr,
        output_tr=output_tr,
        delt_tr=delt_tr,
        scheduler=scheduler,  
        epochs=epochs,
        patience=patience,
        threshold=threshold,
        print_interval=5000
    )
    if loss_Tay < 10.1:
        t_next_t = model_Base(temp_val, supply_val, air_val)
        current_mse_score = mean_squared_error(output_val.to("cpu").detach().numpy(), t_next_t.to("cpu").detach().numpy())
        if current_mse_score < min(mse):
            torch.save(model_Base.state_dict(), f"model_Base_t_5_min_HVAC.pth")
            mse.append(current_mse_score)
            print(current_mse_score)

Epoch [1/25000], Loss: 0.059127338
Epoch [5001/25000], Loss: 0.000265864
Epoch [10001/25000], Loss: 0.000265864
Epoch [15001/25000], Loss: 0.000265864
Epoch [20001/25000], Loss: 0.000265864
0.0002998158
Epoch [1/25000], Loss: 0.030136772
Early stopping at epoch 1274 due to no significant improvement.
Epoch [1/25000], Loss: 0.824278593
Early stopping at epoch 3668 due to no significant improvement.
Epoch [1/25000], Loss: 0.118512243
Epoch [5001/25000], Loss: 0.000262773
Epoch [10001/25000], Loss: 0.000262773
Epoch [15001/25000], Loss: 0.000262773
Epoch [20001/25000], Loss: 0.000262773
0.00029576398
Epoch [1/25000], Loss: 0.018264063
Epoch [5001/25000], Loss: 0.000265873
Epoch [10001/25000], Loss: 0.000265864
Epoch [15001/25000], Loss: 0.000265864
Epoch [20001/25000], Loss: 0.000265864
Epoch [1/25000], Loss: 0.053125404
Epoch [5001/25000], Loss: 0.000262773
Epoch [10001/25000], Loss: 0.000262773
Epoch [15001/25000], Loss: 0.000262773
Epoch [20001/25000], Loss: 0.000262775
Epoch [1/25000]

In [ ]:
def evaluate_model(model_path, model_class, num_predict, device='cpu'):
    r2_loop = []
    mse_loop = []
    model = model_class().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    t_next = model(temp_t[0:-num_predict], supply_t[0:-num_predict], air_t[0:-num_predict])
    r2 = r2_score(output_t[:-num_predict].to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
    mse = mean_squared_error(output_t[:-num_predict].to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
    r2_loop.append(r2)
    mse_loop.append(mse)
    for i in range(1, num_predict):
        t_next = model(t_next, supply_t[i:i-num_predict], air_t[i:i-num_predict])
        r2 = r2_score(output_t[i:i-num_predict].to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
        mse = mean_squared_error(output_t[i:i-num_predict].to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
        r2_loop.append(r2)
        mse_loop.append(mse)
    return r2_loop, mse_loop

In [ ]:
N = 100
results_1, results_2 = evaluate_model(model_path=f"model_Base_t_5_min_HVAC.pth", model_class=Base_Net, num_predict=7, device=device)
for value in results_1:
    print(value.round(4))


0.9303
0.794
0.627
0.4507
0.2858
0.1286
-0.0067


In [34]:
for value in results_2:
    print((np.sqrt(value)*7).round(5))

0.24028
0.40807
0.5439
0.65378
0.73881
0.80937
0.86343
